In [1]:
# Deep Learning
# Let's build a spam clasifier

import pandas as pd

dataset = pd.read_csv('spam.csv', encoding='latin-1')

print(dataset.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [2]:
# Fill null values with empty strings
dataset.fillna(value='', inplace=True)


In [3]:
# score phrase and title
print(dataset.columns.values)

['v1' 'v2' 'Unnamed: 2' 'Unnamed: 3' 'Unnamed: 4']


In [4]:
# Extract the required columns for inputs and outputs
totalX = dataset.v2 # SMS text
totalY = dataset.v1 # spam or ham

In [5]:
import tflearn
from tflearn.data_utils import pad_sequences, VocabularyProcessor

import numpy as np

# Convert the strings in the input into integers corresponding to the dictionary positions
# Data is automatically padded so we need to pad_sequences manually
vocab_proc = VocabularyProcessor(30) # max document length
totalX = np.array(list(vocab_proc.fit_transform(totalX)))

In [6]:
totalX

array([[    1,     2,     3, ...,     0,     0,     0],
       [   21,    22,    23, ...,     0,     0,     0],
       [   27,    28,     8, ...,    49,    50,    51],
       ..., 
       [11153,   415,     8, ...,     0,     0,     0],
       [  174,  3484,   286, ...,     0,     0,     0],
       [ 4643,   660,  1641, ...,     0,     0,     0]])

In [7]:
# We will have 2 classes in total for prediction, indices from 0 to 1
vocab_proc2 = VocabularyProcessor(1)
totalY = np.array(list(vocab_proc2.fit_transform(totalY))) - 1

In [8]:
totalY

array([[0],
       [0],
       [1],
       ..., 
       [0],
       [0],
       [0]])

In [9]:
from keras.utils.np_utils import to_categorical

# here totalY is numbered dictionary entries (0, 1, ... to 10)
totalY[5]
# Convert the indices into 11 dimensional vectors
totalY = to_categorical(totalY, 2)
# here totalY is a binary matrix
totalY

Using TensorFlow backend.


array([[ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [10]:
from sklearn.cross_validation import train_test_split

# Split into training and testing data
trainX, testX, trainY, testY = train_test_split(totalX, totalY, test_size=0.1)

print(trainX[0])
print(testX[0])
print(trainY[0])
print(testY[0])

[2019 3741 1053  184  182  384 1754  168  590    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
[2678  185  538  105 1149    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
[ 1.  0.]
[ 1.  0.]


/Users/miquel/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
import tensorflow as tf
tf.reset_default_graph()

# Network building
# 15 words max, so 15 input data
net = tflearn.input_data([None, 30])

# dictionary has 20k words max
# Turns positive integers (indexes) into dense vectors of fixed size.
net = tflearn.embedding(net, input_dim=20000, output_dim=128)

# Long Short Term Memory Recurrent Layer.
# Each input would have a size of i15x128 and each of these 128
# sized vectors are fed into the LSTM layer one at a time.
# All the intermediate outputs are collected and then passed on to the second LSTM layer.
net = tflearn.lstm(net, 128, dropout=0.8)

# The output is then sent to a fully connected layer that would give us our final 2 classes
net = tflearn.fully_connected(net, 2, activation='softmax')

# We use the adam optimizer instead of standard SGD since it converges much faster
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

In [12]:


# Training
model = tflearn.DNN(net, tensorboard_verbose=0)

# uncomment to load saved model
# model.load('spam_model.tfl')

# comment to use saved model
model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, batch_size=32)

model.save('spam_model.tfl')

print("done training")

Training Step: 1569  | total loss: 0.00030 | time: 12.726s
| Adam | epoch: 010 | loss: 0.00030 - acc: 1.0000 -- iter: 4992/5014
Training Step: 1570  | total loss: 0.00029 | time: 13.809s
| Adam | epoch: 010 | loss: 0.00029 - acc: 1.0000 | val_loss: 0.10926 - val_acc: 0.9803 -- iter: 5014/5014
--
INFO:tensorflow:/Users/miquel/dev/github/deep-learning/message_clasifier/spam_model.tfl is not in all_model_checkpoint_paths. Manually adding it.
done training


In [53]:
# Test predictions

testIdx = 2

input_array = np.array(list(vocab_proc.fit_transform(np.array([dataset.v2[testIdx]]))))

reverse = np.array(list(vocab_proc.reverse(input_array)))
print(reverse)

prediction = model.predict(input_array)

print("Ham: {0:.2f}".format(prediction[0][0]))
print("Spam: {0:.2f}".format(prediction[0][1]))

[ "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 Text FA to 87121 to receive entry question std txt rate T C's apply"]
Ham: 0.00
Spam: 1.00


In [54]:
user_input = "SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info"
user_input

'SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info'

In [59]:
user_input = "WIN CASH TODAY FREE pounds info"

# Transform manual input into vocabulary numbers
input_array = np.array(list(vocab_proc.fit_transform(np.array([user_input]))))

# reverse to check if the input matches what the dictionary has
reverse = np.array(list(vocab_proc.reverse(input_array)))
print(input_array)
print(reverse)

# perform prediction
prediction = model.predict(input_array)
print("Ham: {0:.2f}".format(prediction[0][0]))
print("Spam: {0:.2f}".format(prediction[0][1]))

[[2311  200 2048  177  205  218    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
[ 'WIN CASH TODAY FREE pounds info <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>']
Ham: 0.00
Spam: 1.00
